# VK-API: GEOtagged photos


by Bella Mironova based on the [materials]("https://gainanov.pro/rus-blog/programming/vk-favourite-places-for-photo/") of Ruslan Gaynanov


## 0. Import libraries


In [30]:
import pandas as pd
import geopandas as gpd
import folium
import requests
from random import randint
from datetime import datetime
from time import sleep, strftime, localtime, mktime, strptime
from csv import DictWriter, DictReader
from folium.plugins import HeatMap

## 1. Select GEOgraphical area and timeperiod


##### Define center coordinates, serach radius and timeperiod


In [72]:
coords = (60.7096, 28.7490)  # coordinates: latitude, longitude

dist = 3000  # search radius in meters

startDate = datetime(2023, 8, 1)  # first date to serach 
endDate = datetime(2023, 9, 1) # last date to serach 

timeperiod = (startDate.timestamp(), endDate.timestamp())

##### Define your vk token and vk version


In [48]:
vk_access_token = '34a9a2a734a9a2a734a9a2a70937bb1d32334a934a9a2a7575af68365af3b1d411ce236' # your unique api key

vk_vers = 5.131 # current vk version

## 2. Search photos in a selected area and timeperiod


In [49]:

def getData(coords, timeperiod, offset):
    params = {
        'lat': coords[0],
        'long': coords[1],
        'count': '1000',
        'offset': offset,
        'radius': dist,
        'start_time': timeperiod[0],
        'end_time': timeperiod[1],
        'access_token': vk_access_token,
        'v': vk_vers,
        'sort': 0 # by date of creation
    }
    return requests.get("https://api.vk.com/method/photos.search",
                        params=params, verify=True).json()

## 3. Save response data


In [92]:
def savePoints(resp, df_points):
    try:
        items = resp['response']['items']
    except KeyError:
        return
    for f in items:
        try:
            df_points.loc[len(df_points.index)] = [f['id'], datetime.fromtimestamp(f['date']),f['text'],f['lat'],  f['long']]
        except KeyError:
            continue

## 4. Getting data and saving it to gdb


In [87]:
df_points = pd.DataFrame(columns = ['id', 'date', 'text', 'lat', 'long'])
step = 24*60*60 # step = 1 day
i = timeperiod[0]

while i < timeperiod[1]:
    resp = getData(coords, (i, i+step), 0)
    savePoints(resp, df_points)
    count = resp['response']['count']
    returned = len(resp['response']['items'])
    if count > returned:
        offset = returned
        while offset < count and offset < 3000:
            resp = getData(coords, (i, i+step), offset)
            savePoints(resp, df_points)
            count = resp['response']['count']
            returned = len(resp['response']['items'])
            offset = offset + returned
            if returned == 0:
                break
    i = i + step
    sleep(0.5)      

In [91]:
map_data

,id,date,text,lat,long
0,457239021,1690923366,,60.713997,28.740599
1,457250509,1690920847,,60.710503,28.734596
2,457240226,1690918621,,60.715806,28.727004
3,457240225,1690918617,,60.715807,28.729391
4,457240224,1690918613,,60.714114,28.736794
...,...,...,...,...,...
3543,457244676,1693467203,,60.710908,28.754069
3544,457239270,1693461892,,60.710721,28.749984
3545,457290400,1693460296,,60.709406,28.758467
3546,457394058,1693431829,,60.714009,28.725766


In [ ]:
map_

In [7]:
save_map_to_csv(map_data, 'vyborg2022_2.csv')

In [68]:
map_df = get_map_to_df(map_data)

In [58]:
map_df.head()

,id,date,lat,long
0,457239021,01.08.2023,60.713997,28.740599
0,457250509,01.08.2023,60.710503,28.734596
0,457240226,01.08.2023,60.715806,28.727004
0,457240225,01.08.2023,60.715807,28.729391
0,457240224,01.08.2023,60.714114,28.736794


In [59]:
spatial = gpd.GeoDataFrame(map_df, geometry=gpd.points_from_xy(map_df.long, map_df.lat), crs="EPSG:4326")

In [69]:
osm_map = folium.Map(location=[map_df.lat.mean(), map_df.long.mean()], zoom_start=10)


In [70]:
map_matrix = map_df[['lat', 'long']].values
HeatMap(map_matrix).add_to(osm_map)

In [71]:
osm_map